<h3>Data preprocessing</h3>

In [4]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)
data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
                                                                  inferschema='true', 
                                                                  quote = '"', 
                                                                  escape = '"',
                                                                 multiline = 'true',
                                                                 ignoreTrailingWhiteSpace = 'true').load('data.csv')

# There were some problems reading the data, here I found the solutions
# https://stackoverflow.com/questions/40413526/reading-csv-files-with-quoted-fields-containing-embedded-commas
#https://stackoverflow.com/questions/50477857/spark-fails-to-read-csv-when-last-column-name-contains-spaces

In [5]:
#read data frame in pandas

import pandas as pd
odf = pd.read_csv('/Users/Gianina/Documents/MSc/spark/notebooks/data.csv', encoding = 'unicode_escape')
odf.head()

,X,book_title,review_title,review_user,book_id,review_id,timestamp,review_text,review_score
0,1,A Gentleman in Moscow: A Novel,Russian aristocracy following the Russian Revo...,Kansabelle,0143110438,R2UFCQ9WES7VFH,1555241537,"A great read. In addition to the plot, charact...",4
1,2,A Gentleman in Moscow: A Novel,Knowing nothing ahead of time just walk into t...,D.P. McHenry,0143110438,R24B1HA9J9I99G,1555241542,"Great story, well told. Characters were well d...",4
2,3,Pet Sematary: A Novel,"One of King's finest, most frightening, & earl...",Gordon Hoffman,198211598X,R1P137WFADSBYR,1555241649,Only the second novel written and published un...,4
3,4,Less (Winner of the Pulitzer Prize): A Novel,Not my favorite,R. Zocher,0316316121,R35533AKR5CBNS,1555242044,This book is t what I expected. Story is a lit...,4
4,5,Supermarket,"AMAZING, BOBBY",D. Mahoney,1982127139,R1D6LXSDR0CRMN,1555242169,As someone who hates reading and never really ...,4


In [6]:
#merge review title and text into a single column in pandas
odf['text'] = odf.agg('{0[review_title]}  {0[review_text]}'.format, axis=1)
odf.head()

,X,book_title,review_title,review_user,book_id,review_id,timestamp,review_text,review_score,text
0,1,A Gentleman in Moscow: A Novel,Russian aristocracy following the Russian Revo...,Kansabelle,0143110438,R2UFCQ9WES7VFH,1555241537,"A great read. In addition to the plot, charact...",4,Russian aristocracy following the Russian Revo...
1,2,A Gentleman in Moscow: A Novel,Knowing nothing ahead of time just walk into t...,D.P. McHenry,0143110438,R24B1HA9J9I99G,1555241542,"Great story, well told. Characters were well d...",4,Knowing nothing ahead of time just walk into t...
2,3,Pet Sematary: A Novel,"One of King's finest, most frightening, & earl...",Gordon Hoffman,198211598X,R1P137WFADSBYR,1555241649,Only the second novel written and published un...,4,"One of King's finest, most frightening, & earl..."
3,4,Less (Winner of the Pulitzer Prize): A Novel,Not my favorite,R. Zocher,0316316121,R35533AKR5CBNS,1555242044,This book is t what I expected. Story is a lit...,4,Not my favorite This book is t what I expecte...
4,5,Supermarket,"AMAZING, BOBBY",D. Mahoney,1982127139,R1D6LXSDR0CRMN,1555242169,As someone who hates reading and never really ...,4,"AMAZING, BOBBY As someone who hates reading a..."


In [7]:
#drop_list = ['Dates', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']
#data = data.select([column for column in data.columns if column not in drop_list])
data.show(5)

+---+--------------------+--------------------+--------------+----------+--------------+----------+--------------------+------------+
|  X|          book_title|        review_title|   review_user|   book_id|     review_id| timestamp|         review_text|review_score|
+---+--------------------+--------------------+--------------+----------+--------------+----------+--------------------+------------+
|  1|A Gentleman in Mo...|Russian aristocra...|    Kansabelle|0143110438|R2UFCQ9WES7VFH|1555241537|A great read. In ...|           4|
|  2|A Gentleman in Mo...|Knowing nothing a...|  D.P. McHenry|0143110438|R24B1HA9J9I99G|1555241542|Great story, well...|           4|
|  3|Pet Sematary: A N...|One of King's fin...|Gordon Hoffman|198211598X|R1P137WFADSBYR|1555241649|Only the second n...|           4|
|  4|Less (Winner of t...|     Not my favorite|     R. Zocher|0316316121|R35533AKR5CBNS|1555242044|This book is t wh...|           4|
|  5|         Supermarket|      AMAZING, BOBBY|    D. Mahoney|

In [9]:
data.printSchema()

root
 |-- X: integer (nullable = true)
 |-- book_title: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_user: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_score: integer (nullable = true)



In [10]:
from pyspark.sql.functions import col

data.groupBy("review_score") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------------+-----+
|review_score|count|
+------------+-----+
|           5| 3236|
|           4|  598|
|           3|  187|
|           1|  110|
|           2|  108|
+------------+-----+



In [11]:
# data.count() #this does not work for me

In [12]:
#data.filter("review_score is NULL").show() # No nulls anymore
#data.filter("X is NULL").show() 
#data.filter("book_title is NULL").show()
#data.filter("review_title is NULL").show()
#data.filter("review_user is NULL").show()
#data.filter("book_id is NULL").show()
#data.filter("review_id is NULL").show()
#data.filter("timestamp is NULL").show()
data.filter("review_text is NULL").show() # There is one null
#data.filter("review_score is NULL").show()

+----+--------------------+--------------------+-----------+----------+--------------+----------+-----------+------------+
|   X|          book_title|        review_title|review_user|   book_id|     review_id| timestamp|review_text|review_score|
+----+--------------------+--------------------+-----------+----------+--------------+----------+-----------+------------+
|1121|Lies My Doctor To...|This book gave a ...|   mawshell|162860378X|R3SIH2LVO3EYMH|1555252626|       null|           5|
+----+--------------------+--------------------+-----------+----------+--------------+----------+-----------+------------+



In [13]:
# Remove observation where review_text is null
data = data.na.drop(subset=["review_text"])

In [14]:
# data.count() # Removed! #this does not work for me

In [15]:
# Concatenate book_title, review_title and review_text into a single column
from pyspark.sql import functions as ff
data = data.withColumn('text', ff.concat(ff.col('review_title'), ff.lit(' '), ff.col('review_text'))) #removed book_title
data.show(1, truncate = False)

+---+------------------------------+----------------------------------------------------+-----------+----------+--------------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|X  |book_title                    |review_title                                        |review_user|book_id   |review_id     |timestamp |review_text                                                                                                                                                                                              

In [16]:
drop_list = ['X', 'book_title', 'review_title', 'review_user', 'book_id', 'review_id', 
             'timestamp', 'review_text']
data = data.select([column for column in data.columns if column not in drop_list])
data.show(20, )

+------------+--------------------+
|review_score|                text|
+------------+--------------------+
|           4|Russian aristocra...|
|           4|Knowing nothing a...|
|           4|One of King's fin...|
|           4|Not my favorite T...|
|           4|AMAZING, BOBBY As...|
|           4|A really fun read...|
|           4|So truly enjoyabl...|
|           4|Well written and ...|
|           3|Not one of his be...|
|           4|Beautiful It's no...|
|           4|The last returns ...|
|           4|Don't open this b...|
|           4|Bobby Bestseller!...|
|           4|Listen to this in...|
|           4|A book with a mes...|
|           4|Glad I read this ...|
|           4|Gentleman From Mo...|
|           4|It is a terrific ...|
|           2|An OK read if the...|
|           4|Just as scary now...|
+------------+--------------------+
only showing top 20 rows



<h3>Now the modelling pipeline starts</h3>
I got it from: https://towardsdatascience.com/multi-class-text-classification-with-pyspark-7d78d022ed35

In [17]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
import nltk
#nltk.download("stopwords")

In [18]:
# set seed for reproducibility
(trainingData, testData) = data.randomSplit([0.8, 0.2], seed = 12345)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 3333
Test Dataset Count: 905


In [19]:
# regular expression tokenizer: To split sentences into words
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

In [20]:
# stop words
stopwordList = nltk.corpus.stopwords.words('english')
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=stopwordList)

In [30]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gianina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=20000, minDF=50)

In [22]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

# Recoding target variable
label_stringIdx = StringIndexer(inputCol = "review_score", outputCol = "label") 
labels_stars = label_stringIdx.fit(trainingData).labels # Save this levels to be able later to transform back

# Create pipeline
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

In [23]:
# Fit the pipeline to training data.
pipelineFit = pipeline.fit(trainingData)
dataset = pipelineFit.transform(trainingData)
#dataset.show(1, truncate = False)
dataset.show(5)

+------------+--------------------+--------------------+--------------------+--------------------+-----+
|review_score|                text|               words|            filtered|            features|label|
+------------+--------------------+--------------------+--------------------+--------------------+-----+
|           1|*Eye roll* I firs...|[eye, roll, i, fi...|[eye, roll, first...|(415,[0,2,3,4,6,9...|  4.0|
|           1|1 of the normal H...|[1, of, the, norm...|[1, normal, start...|(415,[0,16,32,42,...|  4.0|
|           1|580 pages to just...|[580, pages, to, ...|[580, pages, quit...|(415,[0,1,2,3,4,5...|  4.0|
|           1|A political hit j...|[a, political, hi...|[political, hit, ...|(415,[0,22,54,227...|  4.0|
|           1|Amazing Read but ...|[amazing, read, b...|[amazing, read, d...|(415,[0,1,12,25,5...|  4.0|
+------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows



In [24]:
# Fit the pipeline to test data.
test_dataset = pipelineFit.transform(testData)
#test_dataset.show(1, truncate = False)
test_dataset.show(5)

+------------+--------------------+--------------------+--------------------+--------------------+-----+
|review_score|                text|               words|            filtered|            features|label|
+------------+--------------------+--------------------+--------------------+--------------------+-----+
|           1|A waste of money ...|[a, waste, of, mo...|[waste, money, ti...|(415,[5,10,56,375...|  4.0|
|           1|Attn AMAZON Kindl...|[attn, amazon, ki...|[attn, amazon, ki...|(415,[0,11,116,13...|  4.0|
|           1|Baid and Switch I...|[baid, and, switc...|[baid, switch, ha...|(415,[21,23,138,2...|  4.0|
|           1|Boring ! I'm a hu...|[boring, i, m, a,...|[boring, huge, fa...|(415,[2,22,144,30...|  4.0|
|           1|Boring Great plot...|[boring, great, p...|[boring, great, p...|(415,[4,25,63,230...|  4.0|
+------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows



In [25]:
# Original encoding
trainingData.groupBy("review_score") \
    .count() \
    .orderBy(col("count").desc()) \
    .show() 

+------------+-----+
|review_score|count|
+------------+-----+
|           5| 2543|
|           4|  466|
|           3|  146|
|           2|   90|
|           1|   88|
+------------+-----+



In [26]:
# Show how the encoding changed
dataset.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show() 

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 2543|
|  1.0|  466|
|  2.0|  146|
|  3.0|   90|
|  4.0|   88|
+-----+-----+



<h3>Logistic regression using count vector features</h3>

In [28]:
from pyspark.ml.feature import IndexToString

# Transform back from index to original coding
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedScore", labels = labels_stars)

In [71]:
lr = LogisticRegression(maxIter=20, regParam=0, elasticNetParam=0)

lrModel = lr.fit(dataset) # Fit model
predictions = lrModel.transform(test_dataset) # Predict
predictions = labelConverter.transform(predictions) # Transform labels

predictions.filter(predictions['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 20, truncate = 45)

+---------------------------------------------+---------------------------------------------+------------+--------------+
|                                         text|                                  probability|review_score|predictedScore|
+---------------------------------------------+---------------------------------------------+------------+--------------+
|A masterpiece of theory from from a master...|[0.9999999999999991,7.954030889775193E-16,...|           4|             5|
|A Mentally Ill Woman, Anna Fox, Witnesses ...|[0.9999999999996585,3.415128690030579E-13,...|           4|             5|
|All Stiva, No Lev On the plus side, the no...|[0.9999999999981952,1.1338152616723868E-12...|           4|             5|
|exceptional novel - one of the best this y...|[0.9999999999943094,5.690332398125168E-12,...|           5|             5|
|Another blockbuster of a historical fictio...|[0.9999999851303771,1.4869556367930017E-8,...|           5|             5|
|An engaging (exciting, 

In [72]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions) # This is the accuracy

0.7156069853194736

In [64]:
predictions.head(1)

[Row(review_score=1, text='A waste of money and time Too long... too overly dramatic.  Narrator on audio needs to tone it down down down a few notches. Like waiting for paint to dry.', words=['a', 'waste', 'of', 'money', 'and', 'time', 'too', 'long', 'too', 'overly', 'dramatic', 'narrator', 'on', 'audio', 'needs', 'to', 'tone', 'it', 'down', 'down', 'down', 'a', 'few', 'notches', 'like', 'waiting', 'for', 'paint', 'to', 'dry'], filtered=['waste', 'money', 'time', 'long', 'overly', 'dramatic', 'narrator', 'audio', 'needs', 'tone', 'notches', 'like', 'waiting', 'paint', 'dry'], features=SparseVector(415, {5: 1.0, 10: 1.0, 56: 1.0, 375: 1.0, 395: 1.0}), label=4.0, rawPrediction=DenseVector([-29.7877, -31.5579, -31.4234, -30.8975, -29.5145]), probability=DenseVector([0.3324, 0.0566, 0.0647, 0.1096, 0.4368]), prediction=4.0, predictedScore='1')]

<h4> Crossvalidation </h4>

In [65]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

lr = LogisticRegression(maxIter=20, regParam=0, elasticNetParam=0)

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.05, 0.1, 0.2, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.2, 0.4, 0.6, 0.8, 1]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)

cvModel = cv.fit(dataset)

predictions = cvModel.transform(test_dataset)
predictions = labelConverter.transform(predictions) # Transform labels

# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label")
evaluator.evaluate(predictions)

0.7208517548212714

In [66]:
best_model = cvModel.bestModel
best_model

LogisticRegressionModel: uid = LogisticRegression_394821c88d63, numClasses = 5, numFeatures = 415

In [67]:
best_reg_param = best_model._java_obj.getRegParam()
best_elasticnet_param = best_model._java_obj.getElasticNetParam()
print(best_reg_param);print(best_elasticnet_param)

0.01
0.0


In [68]:
predictions.filter(predictions['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 20, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|A Mentally Ill Woman, Anna Fox, Witne...|[0.999999590727291,4.0898569769996767...|           4|             5|
|A masterpiece of theory from from a m...|[0.9999966705537479,2.733524682498626...|           4|             5|
|exceptional novel - one of the best t...|[0.9999966412120348,2.139144520826201...|           5|             5|
|Sometimes the price of justice is a g...|[0.9999697621108411,2.964081361710977...|           5|             5|
|All Stiva, No Lev On the plus side, t...|[0.9999234337172712,8.740677530493783...|           4|             5|
|Rachel Provides Easy Steps For Creati...|[0.9999016220931114,6.666044895464801...|           5|        

In [69]:
# To evaluate and get confusion matrix: https://runawayhorse001.github.io/LearningApacheSpark/classification.html
#from pyspark.mllib.evaluation import MulticlassMetrics
#metrics = MulticlassMetrics(predictionCol="predictions", labelCol="label")
#metrics.evaluate(predictions)

<h3> Naive Bayes</h3>

In [29]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(smoothing=1)

model = nb.fit(dataset)
predictions = model.transform(test_dataset)
predictions = labelConverter.transform(predictions) # Transform labels

predictions.filter(predictions['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|The best book you will ever read! By ...|[0.9999962814618197,3.503214632379414...|           5|             5|
|What a POWERFUL story!! Unplanned is ...|[0.9999770160552786,2.298346047933018...|           5|             5|
|Beautifully written with elegant Beau...|[0.9999733020487866,2.506506401722169...|           4|             5|
|Amazing, beautiful, accessible book! ...|[0.9999701132582072,2.156537250510015...|           5|             5|
|Bravo, Mr. Towles! After reading "Rul...|[0.9999586010516772,1.892012273915955...|           5|             5|
|exceptional novel - one of the best t...|[0.9999424971293328,5.642610535653649...|           5|        

In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.7323989945784909

<h3>Random Forest</h3>

In [76]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)

# Train model with Training Data
rfModel = rf.fit(dataset)
predictions = rfModel.transform(test_dataset)
predictions = labelConverter.transform(predictions) # Transform labels

predictions.filter(predictions['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|Wonderful book!  Must Read! Loved thi...|[0.8020223606805952,0.119060294868325...|           5|             5|
|Loved, loved, loved this book This wa...|[0.7968827288093231,0.121055193727271...|           5|             5|
|Beautifully narrated. Listened via Au...|[0.7967116264758359,0.121800029620569...|           5|             5|
|Loved every minute One of the finest,...|[0.7959696472789387,0.120406497649916...|           5|             5|
|Gripping Loved this mystery with a da...|[0.7953760633801066,0.122145671073576...|           5|             5|
|first class book arrived on time, bea...|[0.7950154509843793,0.121142918440051...|           4|        

In [77]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.6626568462275606

<h4> Crossvalidation </h4>

In [78]:
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4)

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [100, 200, 500]) # regularization parameter
             .addGrid(rf.maxDepth, [4, 10, 20]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)

cvModel = cv.fit(dataset)

predictions = cvModel.transform(test_dataset)
predictions = labelConverter.transform(predictions) # Transform labels

# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label")
evaluator.evaluate(predictions)

0.6697956060200818

In [79]:
best_model = cvModel.bestModel
best_model

RandomForestClassificationModel (uid=RandomForestClassifier_5586208643d7) with 500 trees

In [80]:
best_numTrees = best_model.getNumTrees
best_maxDepth = best_model.getOrDefault('maxDepth')
print(best_numTrees);print(best_maxDepth)

500
20


In [81]:
predictions.filter(predictions['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|Amazing tale and well constructed sto...|[0.9087854836787017,0.060941323855234...|           5|             5|
|Wonderful book!  Must Read! Loved thi...|[0.9073353613427794,0.063538289335196...|           5|             5|
|One of the best books I have read thi...|[0.906737923225878,0.0626705543419397...|           5|             5|
|Amazing must read! Great book! I laug...|[0.903109562062987,0.0563759156793555...|           5|             5|
|Best book I have read in a couple of ...|[0.897576428983657,0.0644242280585768...|           5|             5|
|Heartwarming. Well written. Lovely bo...|[0.8973500438077334,0.062417050185536...|           5|        

<h3>TF-IDF Features</h3>

In [32]:
from pyspark.ml.feature import HashingTF, IDF

In [33]:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=20000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms

pipeline_tfidf = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])

In [34]:
# applying pipeline to training data
pipelineFit_tfidf = pipeline_tfidf.fit(trainingData)
dataset_tfidf = pipelineFit_tfidf.transform(trainingData)
dataset_tfidf.show(1)

+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|review_score|                text|               words|            filtered|         rawFeatures|            features|label|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|           1|*Eye roll* I firs...|[eye, roll, i, fi...|[eye, roll, first...|(20000,[456,1305,...|(20000,[456,1305,...|  4.0|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 1 row



In [35]:
# applying pipeline to test data
test_dataset_tfidf = pipelineFit_tfidf.transform(testData)
test_dataset_tfidf.show(1)

+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|review_score|                text|               words|            filtered|         rawFeatures|            features|label|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|           1|A waste of money ...|[a, waste, of, mo...|[waste, money, ti...|(20000,[437,450,1...|(20000,[437,450,1...|  4.0|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 1 row



<h3>Logistic regression TF-IDF</h3>

In [86]:
lr_tfidf = LogisticRegression(maxIter=20, regParam=0, elasticNetParam=0)

lrModel_tfidf = lr_tfidf.fit(dataset_tfidf)
predictions_tfidf = lrModel_tfidf.transform(test_dataset_tfidf)
predictions_tfidf = labelConverter.transform(predictions_tfidf) # Transform labels

predictions_tfidf.filter(predictions_tfidf['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|Beautifully done and down to earth I ...|[1.0,1.0951071313748669E-16,5.0335173...|           5|             5|
|SO Twisty Twisted “I want my kids to ...|[1.0,6.689446406497144E-17,2.61317220...|           5|             5|
|Engrossing story! During the first pa...|[1.0,3.252631500218192E-17,7.17720871...|           5|             5|
|SLOW BUILD TO THE TOP OF THE ROLLER C...|[1.0,3.044315421944498E-17,3.72133604...|           5|             5|
|On the edge of my seat The author rea...|[1.0,2.4405740959251534E-17,2.8124606...|           5|             5|
|Great book! This has been the best bo...|[1.0,1.6857201275336582E-17,4.7220161...|           5|        

In [87]:
evaluator_tfidf = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_tfidf.evaluate(predictions_tfidf)

0.7240072934143862

In [88]:
predictions_tfidf.head(1)

[Row(review_score=1, text='A waste of money and time Too long... too overly dramatic.  Narrator on audio needs to tone it down down down a few notches. Like waiting for paint to dry.', words=['a', 'waste', 'of', 'money', 'and', 'time', 'too', 'long', 'too', 'overly', 'dramatic', 'narrator', 'on', 'audio', 'needs', 'to', 'tone', 'it', 'down', 'down', 'down', 'a', 'few', 'notches', 'like', 'waiting', 'for', 'paint', 'to', 'dry'], filtered=['waste', 'money', 'time', 'long', 'overly', 'dramatic', 'narrator', 'audio', 'needs', 'tone', 'notches', 'like', 'waiting', 'paint', 'dry'], rawFeatures=SparseVector(20000, {437: 1.0, 450: 1.0, 1933: 1.0, 3330: 1.0, 4511: 1.0, 5242: 1.0, 8904: 1.0, 12378: 1.0, 15666: 1.0, 15866: 1.0, 16256: 1.0, 16270: 1.0, 16303: 1.0, 18157: 1.0, 19556: 1.0}), features=SparseVector(20000, {437: 5.547, 450: 4.3507, 1933: 5.9147, 3330: 1.7335, 4511: 4.8538, 5242: 4.1607, 8904: 2.6826, 12378: 5.547, 15666: 3.9688, 15866: 4.6462, 16256: 5.1675, 16270: 0.0, 16303: 0.0, 181

<h4>Cross-validation</h4>

In [89]:
lr_tfidf = LogisticRegression(maxIter=20, regParam=0, elasticNetParam=0)

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr_tfidf.regParam, [0.01, 0.05, 0.1, 0.2, 0.3, 0.5]) # regularization parameter
             .addGrid(lr_tfidf.elasticNetParam, [0.0, 0.2, 0.4, 0.6, 0.8, 1]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())

# Create 5-fold CrossValidator
cv_tfidf = CrossValidator(estimator=lr_tfidf, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)

cvModel_tfidf = cv_tfidf.fit(dataset_tfidf)

predictions_tfidf = cvModel_tfidf.transform(test_dataset_tfidf)
predictions_tfidf = labelConverter.transform(predictions_tfidf) # Transform labels

# Evaluate best model
evaluator_tfidf = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_tfidf.evaluate(predictions_tfidf)

0.7321415751587105

In [90]:
best_model_tfidf = cvModel_tfidf.bestModel
best_model_tfidf

LogisticRegressionModel: uid = LogisticRegression_9dd648d531c8, numClasses = 5, numFeatures = 20000

In [91]:
best_reg_param_tfidf = best_model_tfidf._java_obj.getRegParam()
best_elasticnet_param_tfidf = best_model_tfidf._java_obj.getElasticNetParam()
print(best_reg_param_tfidf);print(best_elasticnet_param_tfidf)

0.01
0.2


In [92]:
predictions_tfidf.filter(predictions_tfidf['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|exceptional novel - one of the best t...|[0.9999622696793031,2.733527368071582...|           5|             5|
|The best book you will ever read! By ...|[0.9999581685305785,1.678374488334822...|           5|             5|
|... beyond five I would choose them e...|[0.9999160145116898,1.548068141767122...|           5|             5|
|An engaging (exciting, even) memoir/m...|[0.9997986893320138,2.072211483093863...|           5|             5|
|Creative, delicious, and the only cak...|[0.9997223337672339,2.508246223168706...|           5|             5|
|Anyone ready to change their life thr...|[0.9997033191053903,2.576619774236201...|           5|        

<h3> Naive Bayes TF-IDF</h3>

In [36]:
nb_tfidf = NaiveBayes(smoothing=1)

model_tfidf = nb_tfidf.fit(dataset_tfidf)
predictions_tfidf = model_tfidf.transform(test_dataset_tfidf)
predictions_tfidf = labelConverter.transform(predictions_tfidf) # Transform labels

predictions_tfidf.filter(predictions_tfidf['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|Holding my breath the whole time! It'...|[1.0,8.91172451906096E-17,9.154199116...|           5|             5|
|A classic for sure Amor Towles has wr...|[1.0,8.651849446427587E-17,1.07908278...|           5|             5|
|Inspiring and fresh approach Inspirin...|[1.0,7.397583925705214E-17,1.22513161...|           5|             5|
|Bye Google! This is a great resource ...|[1.0,5.1578357069290233E-17,5.0991105...|           5|             5|
|This is a unique and wonderful story....|[1.0,4.7054284841733154E-17,3.2240275...|           4|             5|
|A Favorite I absolutely was entranced...|[1.0,4.3282389009190437E-17,7.1926709...|           5|        

In [37]:
evaluator_tfidf = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_tfidf.evaluate(predictions_tfidf)

0.7097232536571314

<h3>Random Forest TF-IDF</h3>

In [38]:
from pyspark.ml.classification import RandomForestClassifier

rf_tfidf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)

# Train model with Training Data
rfModel_tfidf = rf_tfidf.fit(dataset_tfidf)
predictions_tfidf = rfModel_tfidf.transform(test_dataset_tfidf)
predictions_tfidf = labelConverter.transform(predictions_tfidf) # Transform labels

predictions_tfidf.filter(predictions_tfidf['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|Can I Give It 10 Stars? What an amazi...|[0.7773785610197007,0.132148672732905...|           5|             5|
|One of the best books I have read thi...|[0.7772982631687868,0.131864404793228...|           5|             5|
|Five Stars I LOVE this book by Dr Ber...|[0.7772400127825242,0.131678944542849...|           5|             5|
|Five Stars I love this book, Dr. Berr...|[0.7772400127825242,0.131678944542849...|           5|             5|
|Amazing! A must read! As a mother of ...|[0.7767945295209246,0.132840039242313...|           5|             5|
|Amazing must read! Great book! I laug...|[0.7766389580018656,0.132558544803205...|           5|        

In [39]:
evaluator_tfidf = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_tfidf.evaluate(predictions_tfidf)

0.6626568462275606

<h4> Crossvalidation </h4>

In [41]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

rf_tfidf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4)

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(rf_tfidf.numTrees, [100, 200, 500]) # regularization parameter
             .addGrid(rf_tfidf.maxDepth, [4, 10, 20]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())

# Create 5-fold CrossValidator
cv_tfidf = CrossValidator(estimator=rf_tfidf, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)

cvModel_tfidf = cv_tfidf.fit(dataset_tfidf)

predictions_tfidf = cvModel_tfidf.transform(test_dataset_tfidf)
predictions_tfidf = labelConverter.transform(predictions_tfidf) # Transform labels

# Evaluate best model
evaluator_tfidf = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label")
evaluator_tfidf.evaluate(predictions_tfidf)

0.6652653733781968

In [42]:
best_model_tfidf = cvModel_tfidf.bestModel
best_model_tfidf

RandomForestClassificationModel (uid=RandomForestClassifier_351e11f29ab0) with 500 trees

In [43]:
best_numTrees_tfidf = best_model_tfidf.getNumTrees
best_maxDepth_tfidf = best_model_tfidf.getOrDefault('maxDepth')
print(best_numTrees_tfidf);print(best_maxDepth_tfidf)

500
20


In [44]:
predictions_tfidf.filter(predictions_tfidf['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|Amazing must read! Great book! I laug...|[0.8394878420017327,0.098090323020065...|           5|             5|
|Wonderful book!  Must Read! Loved thi...|[0.8350624376964675,0.103200703017782...|           5|             5|
|Amazing tale and well constructed sto...|[0.8340589625245075,0.101695406571931...|           5|             5|
|One of the best books I have read thi...|[0.8300513716897474,0.105026777976361...|           5|             5|
|Loved, loved, loved this book This wa...|[0.8277551885868162,0.106696788690534...|           5|             5|
|Loved this book! Loved this book! It'...|[0.8261263650670962,0.105873147661166...|           5|        

<h3> Word2Vec </h3>

In [45]:
from pyspark.ml.feature import Word2Vec

w2v = Word2Vec(vectorSize=3, minCount=0, inputCol="filtered", outputCol="features")

In [46]:
pipeline_w2v = Pipeline(stages=[regexTokenizer, stopwordsRemover, w2v, label_stringIdx])

In [47]:
# applying pipeline to training data
pipelineFit_w2v = pipeline_w2v.fit(trainingData)
dataset_w2v = pipelineFit_w2v.transform(trainingData)
dataset_w2v.show(1)

+------------+--------------------+--------------------+--------------------+--------------------+-----+
|review_score|                text|               words|            filtered|            features|label|
+------------+--------------------+--------------------+--------------------+--------------------+-----+
|           1|*Eye roll* I firs...|[eye, roll, i, fi...|[eye, roll, first...|[0.12171184380861...|  4.0|
+------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 1 row



In [48]:
# applying pipeline to test data
test_dataset_w2v = pipelineFit_w2v.transform(testData)
test_dataset_w2v.show(1)

+------------+--------------------+--------------------+--------------------+--------------------+-----+
|review_score|                text|               words|            filtered|            features|label|
+------------+--------------------+--------------------+--------------------+--------------------+-----+
|           1|A waste of money ...|[a, waste, of, mo...|[waste, money, ti...|[0.12965251902739...|  4.0|
+------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 1 row



<h3>Logistic regression Word2Vec</h3>

In [49]:
lr_w2v = LogisticRegression(maxIter=20, regParam=0, elasticNetParam=0)

lrModel_w2v = lr_w2v.fit(dataset_w2v)
predictions_w2v = lrModel_w2v.transform(test_dataset_w2v)
predictions_w2v = labelConverter.transform(predictions_w2v) # Transform labels

predictions_w2v.filter(predictions_w2v['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|                        Five Stars GREAT|[0.9990179946518062,8.926592373259777...|           5|             5|
|                      Five Stars Thanks!|[0.9989624988812209,9.030248418269357...|           5|             5|
|                     Five Stars loved it|[0.9988892388307916,0.001030702828146...|           5|             5|
|                    Five Stars good book|[0.9938376445727559,0.004656102111888...|           5|             5|
|Five Stars Informative and great writing|[0.9916983515107264,0.007132063323189...|           5|             5|
|Five Stars It doesn't get any better ...|[0.9915531519304558,0.006641721701242...|           5|        

In [50]:
predictions_w2v.head(1)

[Row(review_score=1, text='A waste of money and time Too long... too overly dramatic.  Narrator on audio needs to tone it down down down a few notches. Like waiting for paint to dry.', words=['a', 'waste', 'of', 'money', 'and', 'time', 'too', 'long', 'too', 'overly', 'dramatic', 'narrator', 'on', 'audio', 'needs', 'to', 'tone', 'it', 'down', 'down', 'down', 'a', 'few', 'notches', 'like', 'waiting', 'for', 'paint', 'to', 'dry'], filtered=['waste', 'money', 'time', 'long', 'overly', 'dramatic', 'narrator', 'audio', 'needs', 'tone', 'notches', 'like', 'waiting', 'paint', 'dry'], features=DenseVector([0.1297, -0.0, -0.1009]), label=4.0, rawPrediction=DenseVector([2.2142, 0.4116, -0.6994, -1.0434, -0.883]), probability=DenseVector([0.7676, 0.1266, 0.0417, 0.0295, 0.0347]), prediction=0.0, predictedScore='5')]

In [51]:
evaluator_w2v = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_w2v.evaluate(predictions_w2v)

0.6621138481563025

<h4>Cross-validation</h4>

In [52]:
lr_w2v = LogisticRegression(maxIter=20, regParam=0, elasticNetParam=0)

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr_w2v.regParam, [0.01, 0.05, 0.1, 0.2, 0.3, 0.5]) # regularization parameter
             .addGrid(lr_w2v.elasticNetParam, [0.0, 0.2, 0.4, 0.6, 0.8, 1]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())

# Create 5-fold CrossValidator
cv_w2v = CrossValidator(estimator=lr_w2v, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)

cvModel_w2v = cv_w2v.fit(dataset_w2v)

predictions_w2v = cvModel_w2v.transform(test_dataset_w2v)
predictions_w2v = labelConverter.transform(predictions_w2v) # Transform labels

# Evaluate best model
evaluator_w2v = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_w2v.evaluate(predictions_w2v)

0.6626568462275606

In [56]:
best_model_w2v = cvModel_w2v.bestModel
best_model_w2v

LogisticRegressionModel: uid = LogisticRegression_962370e30246, numClasses = 5, numFeatures = 3

In [57]:
best_reg_param_w2v = best_model_w2v._java_obj.getRegParam()
best_elasticnet_param_w2v = best_model_w2v._java_obj.getElasticNetParam()
print(best_reg_param_w2v);print(best_elasticnet_param_w2v)

0.01
0.0


In [58]:
predictions_w2v.filter(predictions_w2v['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|                        Five Stars GREAT|[0.9966192352581656,0.002631490546815...|           5|             5|
|                     Five Stars loved it|[0.9963680578485298,0.002971471338022...|           5|             5|
|                      Five Stars Thanks!|[0.9962178764782932,0.002683675545206...|           5|             5|
|                    Five Stars good book|[0.9857984657457295,0.008851949287828...|           5|             5|
|Five Stars It doesn't get any better ...|[0.9814926845062876,0.012268472772541...|           5|             5|
|Five Stars Informative and great writing|[0.9813553638420999,0.013978208821741...|           5|        

<h3> Naive Bayes Word2Vec</h3>

In [59]:
nb_w2v = NaiveBayes(smoothing=1)

model_w2v = nb_w2v.fit(dataset_w2v)
predictions_w2v = model_w2v.transform(test_dataset_w2v)
predictions_w2v = labelConverter.transform(predictions_w2v) # Transform labels

predictions_w2v.filter(predictions_w2v['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

Py4JJavaError: An error occurred while calling o49027.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8400.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8400.0 (TID 8996, localhost, executor driver): java.lang.IllegalArgumentException: requirement failed: Naive Bayes requires nonnegative feature values but found [0.12171184380861873,-0.0021886859755554443,-0.12637901212069755].
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.classification.NaiveBayes$.requireNonnegativeValues(NaiveBayes.scala:235)
	at org.apache.spark.ml.classification.NaiveBayes$$anonfun$trainWithLabelCheck$1$$anonfun$4.apply(NaiveBayes.scala:144)
	at org.apache.spark.ml.classification.NaiveBayes$$anonfun$trainWithLabelCheck$1$$anonfun$4.apply(NaiveBayes.scala:144)
	at org.apache.spark.ml.classification.NaiveBayes$$anonfun$trainWithLabelCheck$1$$anonfun$7.apply(NaiveBayes.scala:168)
	at org.apache.spark.ml.classification.NaiveBayes$$an

In [ ]:
evaluator_w2v = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_w2v.evaluate(predictions_w2v)

<h3>Random Forest Word2Vec</h3>

In [60]:
rf_w2v = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)

# Train model with Training Data
rfModel_w2v = rf_w2v.fit(dataset_w2v)
predictions_w2v = rfModel_w2v.transform(test_dataset_w2v)
predictions_w2v = labelConverter.transform(predictions_w2v) # Transform labels

predictions_w2v.filter(predictions_w2v['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|Five Stars Recipes are great and easy...|[0.8571389889976624,0.086616307675158...|           5|             5|
|                     Five Stars loved it|[0.8571389889976624,0.086616307675158...|           5|             5|
|                      Five Stars Thanks!|[0.8571389889976624,0.086616307675158...|           5|             5|
|                        Five Stars GREAT|[0.8571389889976624,0.086616307675158...|           5|             5|
|Five Stars Informative and great writing|[0.8571389889976624,0.086616307675158...|           5|             5|
|Five Stars It doesn't get any better ...|[0.8571389889976624,0.086616307675158...|           5|        

In [61]:
evaluator_w2v = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_w2v.evaluate(predictions_w2v)

0.6626568462275606

<h4> Crossvalidation </h4>

In [63]:
rf_w2v = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4)

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(rf_w2v.numTrees, [100, 200, 500]) # regularization parameter
             .addGrid(rf_w2v.maxDepth, [4, 10, 20]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())

# Create 5-fold CrossValidator
cv_w2v = CrossValidator(estimator=rf_w2v, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)

cvModel_w2v = cv_w2v.fit(dataset_w2v)

predictions_w2v = cvModel_w2v.transform(test_dataset_w2v)
predictions_w2v = labelConverter.transform(predictions_w2v) # Transform labels

# Evaluate best model
evaluator_w2v = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_w2v.evaluate(predictions_w2v)

0.6626568462275606

In [64]:
best_model_w2v = cvModel_w2v.bestModel
best_model_w2v

RandomForestClassificationModel (uid=RandomForestClassifier_37eb3a9816a4) with 100 trees

In [65]:
best_numTrees_w2v = best_model_w2v.getNumTrees
best_maxDepth_w2v = best_model_w2v.getOrDefault('maxDepth')
print(best_numTrees_w2v);print(best_maxDepth_w2v)

100
4


In [66]:
predictions_w2v.filter(predictions_w2v['prediction'] == 0) \
    .select("text","probability","review_score","predictedScore") \
    .orderBy("probability", ascending=False) \
    .show(n = 15, truncate = 40)

+----------------------------------------+----------------------------------------+------------+--------------+
|                                    text|                             probability|review_score|predictedScore|
+----------------------------------------+----------------------------------------+------------+--------------+
|Five Stars Recipes are great and easy...|[0.8571389889976624,0.086616307675158...|           5|             5|
|                     Five Stars loved it|[0.8571389889976624,0.086616307675158...|           5|             5|
|                      Five Stars Thanks!|[0.8571389889976624,0.086616307675158...|           5|             5|
|                        Five Stars GREAT|[0.8571389889976624,0.086616307675158...|           5|             5|
|Five Stars Informative and great writing|[0.8571389889976624,0.086616307675158...|           5|             5|
|Five Stars It doesn't get any better ...|[0.8571389889976624,0.086616307675158...|           5|        

In [67]:
predictions_w2v.head(1)

[Row(review_score=1, text='A waste of money and time Too long... too overly dramatic.  Narrator on audio needs to tone it down down down a few notches. Like waiting for paint to dry.', words=['a', 'waste', 'of', 'money', 'and', 'time', 'too', 'long', 'too', 'overly', 'dramatic', 'narrator', 'on', 'audio', 'needs', 'to', 'tone', 'it', 'down', 'down', 'down', 'a', 'few', 'notches', 'like', 'waiting', 'for', 'paint', 'to', 'dry'], filtered=['waste', 'money', 'time', 'long', 'overly', 'dramatic', 'narrator', 'audio', 'needs', 'tone', 'notches', 'like', 'waiting', 'paint', 'dry'], features=DenseVector([0.1297, -0.0, -0.1009]), label=4.0, rawPrediction=DenseVector([73.4686, 15.3011, 4.9771, 3.1385, 3.1147]), probability=DenseVector([0.7347, 0.153, 0.0498, 0.0314, 0.0311]), prediction=0.0, predictedScore='5')]